# IDR0021 - ROIs

A simple example which reads the 'batch_roi_export' table, performs some basic statistics on it and creates a plot (which should be similar to [Figure 1](https://www.nature.com/articles/ncb2591/figures/1) in the paper ['Subdiffraction imaging of centrosomes reveals higher-order organizational features of pericentriolar material'](https://doi.org/10.1038/ncb2591)). The plot and results are then uploaded to the server and attached to the project.

## Load the romero.gateway library and connect to the server

In [ ]:
options(warn=-1) # Disable warnings to suppress unnecessary output

library(romero.gateway)

user_name = readline('Username: ')
user_password <- getPass::getPass('OMERO password: ')
server <- OMEROServer(host = 'outreach.openmicroscopy.org', port = 4064L, username = user_name, password = user_password)
server <- connect(server)
paste('Successfully logged in as', server@user$getUserName())

## Load the data from the 'batch_roi_export' table as R dataframe

In [ ]:
# Find the project and table file
projectName <- 'idr0021'
tableName <- 'Summary_from_Fiji'

searchResult <- searchFor(server, Project, 'Name', projectName)
project <- searchResult[[1]]

annos <- getAnnotations(server, 'ProjectData', getOMEROID(project), nameFilter = tableName)
annotationFileID = as.integer(annos$FileID)


# Load the table (column 14 and 15, Dataset and bouding_box) directly as R dataframe
df <- loadDataframe(project, annotationFileID, columns=c(14, 15))
df

## Prepare the data

In [ ]:
# CEP120 is represented by two datasets'CEP120/20111106' and 'CEP120/20111209',
# just combine them to 'CEP120'
df$Dataset <- as.character(df$Dataset)
df[ df == 'CEP120/20111106' ] <- 'CEP120'
df[ df == 'CEP120/20111209' ] <- 'CEP120'
df$Dataset <- as.factor(df$Dataset)


# Use the bounding box of the biggest shapes for the specified channel
df$diameter <- sqrt(df$Bounding_Box)

## Plot the data

In [ ]:
# Order the data from lowest to highest mean diameter (to match the order of figure 1)
ag <-aggregate(df$diameter ~ df$Dataset, df, mean)
orderedDatasets <- factor(df$Dataset, levels=ag[order(ag$`df$diameter`), 'df$Dataset'])

plot(df$diameter ~ orderedDatasets, ylab='Bounding Box', xlab="Protein", cex.axis=0.5)

## Is there a significant difference between the proteins?

In [ ]:
# One-way analysis of variance:
fit <- aov(df$diameter ~ df$Dataset)
summary(fit)

# Two-sample Wilcoxon test ('Mann-Whitney') of all pairwise combinations:
combins <- combn(levels(df$Dataset), 2)
params_list <- split(as.vector(combins), rep(1:ncol(combins), each = nrow(combins)))
testResults <- data.frame()
for (param in params_list) {
  testdf <- subset(df, df$Dataset %in% param)
  pval <- wilcox.test(formula = diameter ~ Dataset, data = testdf)$p.value
  testResults<-rbind(testResults, data.frame(Protein_1=param[1], Protein_2=param[2], p_value=pval))
}
testResults

## Attach the results to OMERO project and close connection

In [ ]:
# The plot:
tmpfile <- "/tmp/Bounding_Box_By_Protein.png"
png(tmpfile, width = 4, height= 2.5, units = "in", res = 300, pointsize = 6)
plot(df$diameter ~ orderedDatasets, ylab='Bounding Box', xlab="Protein", cex.axis=0.5)
dev.off()
invisible(attachFile(project, tmpfile)) # Wrapped in invisible(...) only to suppress unnecessary output


# The results from the Mann-Whitney test as OMERO.table (HDF)
invisible(attachDataframe(project, testResults, "Mann-Whitney-pValues"))

# and as CSV file:
tmpfile <- "/tmp/Mann-Whitney-pValues.csv"
write.csv(testResults, file = tmpfile)
invisible(attachFile(project, tmpfile)) 

# Finally disconnect again
invisible(disconnect(server))